In [1]:
import os
import pickle
import models

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from numpyro.diagnostics import hpdi

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from hbmep.config import Config
from hbmep.model.utils import Site as site
from scipy import stats

from models import NonHierarchicalBayesianModel

In [2]:
src = "/home/andres/repos/rat-mapping-paper/reports/L_CIRC/inference.pkl"

with open(src, "rb") as f:
    df, encoder_dict, model, posterior_samples, = pickle.load(f)


prediction_df = model.make_prediction_dataset(df=df, min_intensity=0., max_intensity=500.)
posterior_predictive = model.predict(
    df=prediction_df, posterior_samples=posterior_samples
)


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
src = os.path.join("/home/andres/repos/rat-mapping-paper/reports/L_CIRC/predictions.pkl")

with open(src, "rb") as f:
    x, orig_y, subjects, compound_positions, configurations, response, = pickle.load(f)

y = orig_y

In [4]:
y.shape

(8, 21, 3, 400, 6, 1000)

In [ ]:

for s in subjects:
    
c2 = 